1. 문서의 내용을 읽는다 
2. 문서를 쪼갠다 
   - 토큰 수 초과로 답변을 생성하지 못할 수 있고 
   - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림 
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


1. 문서 쪼개기 

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

2. 문서 임베딩 

In [2]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model="solar-embedding-1-large")

3. 임베딩한 문서 chroma로 저장 

In [3]:
from langchain_pinecone import PineconeVectorStore

#데이터를 처음 저장할 때 
index_name = 'tax-upstage-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

4. 답변 생성을 위한 Retrieval

In [4]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [5]:
retrieved_docs

[Document(metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 초과\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제

In [6]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()


In [7]:
from langchain import hub 
prompt = hub.pull('rlm/rag-prompt')

d:\Anaconda\envs\condatest\lib\site-packages\langsmith\client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
d:\Anaconda\envs\condatest\lib\site-packages\langsmith\client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

5. 답변 생성 

In [8]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [9]:
ai_message = qa_chain({"query":query})

C:\Users\Master\AppData\Local\Temp\ipykernel_6884\980961933.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query":query})


In [10]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 1,536,000원입니다.'}

'result': '연봉 5천만원인 직장인의 소득세는 1,536,000원입니다.' >> 잘못된 답변 

- 표에 있는 내용을 파싱하지 못함